<a href="https://colab.research.google.com/github/hndrr/colab/blob/main/facedetector_ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install gradio
!pip install insightface
!pip install opencv-python-headless

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.6/16.6 MB 27.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 6.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 305.2/305.2 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 381.9/381.9 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 35.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.5/60.5 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.4 MB/s eta 

In [ ]:
!pip install onnxruntime

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.4/6.4 MB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 10.8 MB/s eta 0:00:00


In [ ]:
import gradio as gr
import cv2
import numpy as np
import tempfile
from PIL import Image
from zipfile import ZipFile
import insightface
from insightface.app import FaceAnalysis


In [ ]:
# InsightFaceの初期化
app = FaceAnalysis()
app.prepare(ctx_id=0, det_size=(640, 640))


download_path: /root/.insightface/models/buffalo_l


100%|██████████| 281857/281857 [00:06<00:00, 40962.23KB/s]
/usr/local/lib/python3.10/dist-packages/onnxruntime/capi/onnxruntime_inference_collection.py:69: UserWarning: Specified provider 'CUDAExecutionProvider' is not in available provider names.Available providers: 'AzureExecutionProvider, CPUExecutionProvider'
  warnings.warn(


Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/1k3d68.onnx landmark_3d_68 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/2d106det.onnx landmark_2d_106 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/det_10g.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/genderage.onnx genderage ['None', 3, 96, 96] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/w600k_r50.onnx recognition ['None', 3, 112, 112] 127.5 127.5
set det-size: (640, 640)


In [ ]:
def make_square_bbox(bbox):
    x1, y1, x2, y2 = bbox
    w = x2 - x1
    h = y2 - y1
    side_len = max(w, h)
    center_x, center_y = x1 + w // 2, y1 + h // 2
    new_x1 = max(center_x - side_len // 2, 0)
    new_y1 = max(center_y - side_len // 2, 0)
    new_x2 = new_x1 + side_len
    new_y2 = new_y1 + side_len
    return [new_x1, new_y1, new_x2, new_y2]

def resize_image(img, target_size):
    return cv2.resize(img, (target_size, target_size), interpolation=cv2.INTER_AREA)

def detect_and_crop_faces(file_list):
    with tempfile.TemporaryDirectory() as tmp_dir:
        face_count = 0
        for file_path in file_list:
            img = cv2.imread(file_path)
            faces = app.get(img)

            for face in faces:
                bbox = make_square_bbox(face['bbox'])
                x1, y1, x2, y2 = map(int, bbox)
                face_img = img[y1:y2, x1:x2]
                face_img = resize_image(face_img, 500)
                pil_face = Image.fromarray(cv2.cvtColor(face_img, cv2.COLOR_BGR2RGB))
                pil_face.save(f'{tmp_dir}/face_{face_count}.png')
                face_count += 1

        if face_count == 0:
            return "顔が検出されませんでした。", None

        zip_buffer = tempfile.NamedTemporaryFile(delete=False)
        with ZipFile(zip_buffer.name, 'w') as zipf:
            for i in range(face_count):
                zipf.write(f'{tmp_dir}/face_{i}.png', f'face_{i}.png')

        return "顔検出完了。ZIPファイルをダウンロードしてください。", zip_buffer.name

def clear_inputs():
    return [], "入力をクリアしました。", None  # ファイル入力を空のリストでリセット


In [ ]:
with gr.Blocks() as demo:
    with gr.Row():
        with gr.Column():
            file_input = gr.File(file_types=["image"], file_count="multiple", label="Upload Image Files")
            clear_button = gr.Button("クリア")
            submit_button = gr.Button("顔検出", variant="primary")
        with gr.Column():
            output_label = gr.Label(value="結果表示")
            download_link = gr.File(label="Download Zip")

    file_input.change(detect_and_crop_faces, inputs=[file_input], outputs=[output_label, download_link])
    submit_button.click(detect_and_crop_faces, inputs=[file_input], outputs=[output_label, download_link])
    clear_button.click(clear_inputs, outputs=[file_input, output_label, download_link])

demo.launch()


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://48bfad495515124a10.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
